# Working with Spans

In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

# If running from github repo, can use this:
repo = Path().cwd().parent.parent.resolve()
sys.path.append(str(repo))

In [2]:
from pprint import pformat
from pprint import pprint

from examples.expositional.end2end_apps.custom_app.custom_app import CustomApp
import pandas as pd

from trulens_eval import instruments
from trulens_eval.trace.category import Categorizer
from trulens_eval.tru_custom_app import TruCustomApp

In [3]:
# Create custom app:
ca = CustomApp(delay=0.0, alloc=0)

# Create trulens wrapper:
ta = TruCustomApp(
    ca,
    app_id="customapp",
)

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


In [4]:
instruments.Instrument().print_instrumentation()

Module examples.expositional.end2end_apps.custom_app.custom_app*
  Class examples.expositional.end2end_apps.custom_app.custom_app.CustomApp
    Method retrieve_chunks: (self, data)
    Method respond_to_query: (self, input)
    Method arespond_to_query: (self, input)
  Class examples.expositional.end2end_apps.custom_app.custom_app.CustomTemplate
    Method fill: (self, question, answer)

Module examples.expositional.end2end_apps.custom_app.custom_llm*
  Class examples.expositional.end2end_apps.custom_app.custom_llm.CustomLLM
    Method generate: (self, prompt: str)

Module examples.expositional.end2end_apps.custom_app.custom_memory*
  Class examples.expositional.end2end_apps.custom_app.custom_memory.CustomMemory
    Method remember: (self, data: str)

Module examples.expositional.end2end_apps.custom_app.custom_retriever*
  Class examples.expositional.end2end_apps.custom_app.custom_retriever.CustomRetriever
    Method retrieve_chunks: (self, data)
      Span type: SpanType.RETRIEVER

Mo

In [5]:
with ta as recorder:
    res = ca.respond_to_query(f"hello")

rec = recorder.get()

In [7]:
spans = Categorizer.spans_of_record(rec)

pd.DataFrame([(s.name, s.span_type, s.span_id, s.parent_span_id, s.attributes) for s in spans])


,0,1,2,3,4
0,root,SpanType.ROOT,12515400569902585719,NaN,{'trulens_eval@span_type': 'SpanRoot'}
1,retrieve_chunks,SpanType.RETRIEVER,10098215340056937626,1.251540e+19,"{'trulens_eval@span_type': 'SpanRetriever', 't..."
2,retrieve_chunks,SpanType.OTHER,2922204723152669435,1.251540e+19,{'trulens_eval@span_type': 'SpanOther'}
3,remember,SpanType.OTHER,14392575192796042941,1.251540e+19,{'trulens_eval@span_type': 'SpanOther'}
4,generate,SpanType.OTHER,4309257820743789116,1.251540e+19,{'trulens_eval@span_type': 'SpanOther'}
5,fill,SpanType.OTHER,13201551325662865226,1.251540e+19,{'trulens_eval@span_type': 'SpanOther'}
6,remember,SpanType.OTHER,4305599740194858846,1.251540e+19,{'trulens_eval@span_type': 'SpanOther'}
7,respond_to_query,SpanType.OTHER,5226642946292543104,1.251540e+19,{'trulens_eval@span_type': 'SpanOther'}
